<h1> Manual implementation starts here </h1>

<h1> List of Stop Words </h1>

In [38]:
import os
fd = open("D:/document_project/stopwords.txt","r")
strr = ""
stop_words = []
strr = (fd.readlines())
for i in strr:
    stop_words.append(i[0:-1])

<h2> training of data </h2>

In [39]:

import numpy as np
import math
import pandas as pd
path = "D:/document_project/mini_newsgroups"
word_list= {}
file_names = {}
voc = {}
class_counts = {}
for filename in os.listdir(path):
    new_path = os.path.join(path, filename)
    word_list[filename]={}
    
    file_names[filename] = []
    
    class_counts[filename] = 0
    for textfiles in os.listdir(new_path):
        class_counts[filename] +=1
        file_names[filename].append(textfiles)
        file = os.path.join(new_path, textfiles)
        f = open(file,"r")
        for line in f:
            if 'Lines' in line:
                break
        for line in f:
            for word in line.replace(',','').replace('>','').replace(']','').replace('[','').replace(':','').replace('\'','').replace('.','').replace('(','').replace(')','').lower().split():
                if word not in stop_words:
                    if word not in word_list[filename]:
                        word_list[filename][word]=1
                        voc[word]=1
                    elif word in word_list[filename]:
                        word_list[filename][word] +=1
                        voc[word] += 1

        f.close()


<h2> Predict single class probability</h2>

In [50]:
def findSingleClassProb(file,word_list1 , word_list2 , class_counts ,  c):
    ans = 0
    pr_class = class_counts[c]
    ans = pr_class 
    f = open(file,"r")
    
    for line in f:
        if 'Lines' in line:
                break
    for line in f:
        for word in line.replace(',','').replace('>','').replace(']','').replace('[','').replace(':','').replace('\'','').replace('.','').replace('(','').replace(')','').lower().split():
        
            if (word in word_list1[c]):
                
                a = len(word_list2[c].keys())
                b = len(word_list1[c].keys())
                pr_this_feature = (word_list1[c][word] + 1)/(a + b)
                ans =  (pr_this_feature)*(ans)*1000
    return ans

<h2> Predict single file probability</h2>

In [41]:
def predictSingle(file, word_list1 ,word_list2):
    max_pr = 0
    max_class = None
    classes = word_list1.keys()
    for c in classes:
        p = findSingleClassProb(file, word_list1,word_list2 , class_counts, c)
        if p > max_pr:
            max_pr = p
            max_class = c
    return max_class

<h2> function to choose top words and creates two dictionaries</h2>

In [42]:
def top_features(voc , word_list , no_of_features):
    
    class_no = 0
    count_no = 0
    word_list1 = {}
    word_list2 = {}
    
    voc = dict((sorted(voc.items(), key = lambda t:t[1], reverse=True))[0:no_of_features])
    
    
            
    for c in word_list.keys():
        word_list1[c] = {}
        word_list2[c] = {}
        for top_word in  voc.keys():
                

                if top_word in word_list[c].keys():
                    word_list1[c][top_word] = word_list[c][top_word]
                    word_list2[c][top_word] = word_list[c][top_word]

                elif top_word not in word_list[c].keys() :
                    word_list1[c][top_word] = 0 

                
    
    return word_list1 , word_list2
                

<h2> predict function</h2>

In [43]:
def predict(voc , word_list , no_of_features):
    y_true = []
    y_pred = []

    word_list1 , word_list2 = top_features(voc , word_list , no_of_features)
    
    path = "D:/document_project/20_newsgroups"
    for filename in os.listdir(path):
        num = 0
        new_path = os.path.join(path, filename)
        for textfiles in os.listdir(new_path):

            if textfiles not in file_names[filename] and (num < 50):
                y_true.append(filename)
                file = os.path.join(new_path, textfiles)

                y_pred.append( predictSingle(file, word_list1 ,word_list2) )
                num +=1   
    return y_true , y_pred

<h2> score</h2>

In [53]:
from sklearn.metrics import accuracy_score
y_true , y_pred = predict(voc , word_list , 6500)
accuracy_score(y_true , y_pred)
#len(y_true), len(y_pred)
#y_pred

0.475

<h1> Manual implementation ends here</h1>

<h1></h1>
<h1> inbuilt implementation starts here</h1>

In [59]:
voc1 =  dict((sorted(voc.items(), key = lambda t:t[1], reverse=True))[0:6500])

<h2> creating a training dataset in  dataframe</h2>

In [ ]:
x_train = pd.DataFrame(columns=voc1.keys())
y_train = pd.DataFrame()
for filename in os.listdir(path):
    new_path = os.path.join(path, filename)
    for textfiles in os.listdir(new_path):
        file = os.path.join(new_path, textfiles)
        f = open(file,"r")
        single_file_counts = {}
        y_train = y_train.append({0 : filename} , ignore_index=True)
        for line in f:
            if 'Lines' in line:
                break
        for line in f:
            for word in line.replace(',','').replace('>','').replace(']','').replace('[','').replace(':','').replace('\'','').replace('.','').replace('(','').replace(')','').lower().split():
                if word in voc1.keys():
                        if word in single_file_counts.keys():
                            single_file_counts[word] += 1 
                        else :
                            single_file_counts[word] = 0
        x_train = x_train.append(single_file_counts , ignore_index=True)          
                    
                    
                    

<h2> creating a testing dataset in  dataframe</h2>

In [ ]:
path = "D:/document_project/20_newsgroups"
x_test = pd.DataFrame(columns=voc1.keys())
y_test = pd.DataFrame()

for filename in os.listdir(path):
    num = 0
    new_path = os.path.join(path, filename)
    for textfiles in os.listdir(new_path):

        if textfiles not in file_names[filename] and (num < 50):
            y_true.append(filename)
            file = os.path.join(new_path, textfiles)
            f = open(file,"r")
            single_file_counts = {}
            y_test = y_test.append({ 0 : filename } , ignore_index=True)
            for line in f:
                if 'Lines' in line:
                        break
            for line in f:
                for word in line.replace(',','').replace('>','').replace(']','').replace('[','').replace(':','').replace('\'','').replace('.','').replace('(','').replace(')','').lower().split():
                         if word in voc1.keys():
                            if word in single_file_counts.keys():
                                single_file_counts[word] += 1 
                            else :
                                single_file_counts[word] = 0
                                
            x_test = x_test.append(single_file_counts , ignore_index=True)
            num = num + 1

<h2> score</h2>

In [ ]:
from sklearn.naive_bayes import MultinomialNB
mnb = MultinomialNB()
mnb.fit(x_train , y_train)
mnb.score(x_test , y_test)

In [ ]:
x_train.fillna(0, inplace=True)
x_test.fillna(0, inplace=True)

<h1> </h1>
<h1> Inbuilt implemetation ends here</h1>
